# Policy Checker with Azure AI Search

This notebook demonstrates how to use agent operations with the Azure AI Search tool from the Azure Agents service using a synchronous client.

## Setup and Imports

In [1]:
# Import necessary libraries and modules
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole

# Load environment variables from .env file
load_dotenv()

print("✅ Libraries imported and environment variables loaded")

✅ Libraries imported and environment variables loaded


## Configuration

In [11]:
# Retrieve endpoint and model deployment name from environment variables
project_endpoint = "https://msagthack-aifoundry-kl2yy7velhg4u.services.ai.azure.com/api/projects/msagthack-aiproject-kl2yy7velhg4u"  # Ensure the PROJECT_ENDPOINT environment variable is set
model_deployment_name = "gpt-4o-mini"  # Ensure the MODEL_DEPLOYMENT_NAME environment variable is set

# Initialize the AIProjectClient with the endpoint and credentials
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False),  # Use Azure Default Credential for authentication
)

with project_client:
    # Initialize the Azure AI Search tool with the required parameters
    ai_search = AzureAISearchTool(
        index_connection_id=os.environ["AZURE_AI_CONNECTION_ID"],  # Connection ID for the Azure AI Search index
        index_name="insurance-documents-index",  # Name of the search index
        query_type=AzureAISearchQueryType.SIMPLE,  # Query type (e.g., SIMPLE, FULL)
        top_k=3,  # Number of top results to retrieve
        filter="",  # Optional filter for search results
    )

    # Create an agent with the specified model, name, instructions, and tools
    agent = project_client.agents.create_agent(
        model=model_deployment_name,  # Model deployment name
        name="policy-checker",  # Name of the agent
        instructions=
        """"
        You are an expert Insurance Policy Checker Agent specialized in analyzing auto insurance policies and validating claim coverage. Your primary responsibilities include:

        **Core Functions:**
        - Analyze insurance policy documents to determine coverage details
        - Validate if specific claims are covered under policy terms
        - Explain policy limits, deductibles, and exclusions
        - Identify coverage gaps or restrictions
        - Provide clear explanations of policy benefits

        **Policy Types You Handle:**
        - Commercial Auto Policies
        - Comprehensive Auto Policies  
        - High Value Vehicle Policies
        - Liability Only Policies
        - Motorcycle Policies

        **Analysis Guidelines:**
        1. Always reference specific policy sections when providing coverage determinations
        2. Clearly state coverage limits, deductibles, and any applicable restrictions
        3. Identify any exclusions that may apply to the claim
        4. Be precise about effective dates and policy periods
        5. Flag any discrepancies between claim details and policy terms

        **Response Format:**
        - Start with a clear coverage determination (COVERED/NOT COVERED/PARTIAL COVERAGE)
        - Provide the specific policy section reference
        - Explain coverage limits and deductibles
        - List any relevant exclusions or conditions
        - Suggest next steps if coverage issues exist
        - Everything in a clear, concise manner in one paragraph.

        **Tone:** Professional, accurate, and helpful. Always be thorough in your analysis while remaining clear and concise.

        When you cannot find specific information in the policy documents, clearly state what information is missing and what additional documentation would be needed.
        """,  # Instructions for the agent
        tools=ai_search.definitions,  # Tools available to the agent
        tool_resources=ai_search.resources,  # Resources for the tools
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication with the agent
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Send a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,  # ID of the thread
        role=MessageRole.USER,  # Role of the message sender (e.g., user)
        content="What motorcycle coverage options are available?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run in the thread using the tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Log the error if the run fails
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages from the thread
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        # Only show the agent's response, not the user's question
        if message.role == MessageRole.AGENT:
            if message.content and len(message.content) > 0:
                content_item = message.content[0]
                if content_item.get('type') == 'text' and 'text' in content_item:
                    agent_response = content_item['text']['value']
                    print(agent_response)
                    break  # Only show the first agent response

Created agent, ID: asst_aMTmWqCUL9YECCIiEEa7ncYx
Created thread, ID: thread_TjPwTIMlBiLTdtnvrhrsXwYO
Created message, ID: msg_qYZoJEjWAiOzd6Iao7JC9lCJ
Run finished with status: RunStatus.COMPLETED
Motorcycle insurance policies typically offer a variety of coverage options, which may include:

1. **Liability Coverage:** Covers damages to others for which the policyholder is legally responsible, including bodily injury and property damage.
  
2. **Collision Coverage:** Covers damages to the motorcycle resulting from a collision with another vehicle or object, regardless of fault.

3. **Comprehensive Coverage:** Covers non-collision related incidents such as theft, vandalism, natural disasters, and animal strikes.

4. **Uninsured/Underinsured Motorist Coverage:** Provides protection if the motorcycle is damaged by a driver with insufficient insurance or no insurance at all.

5. **Medical Payments Coverage:** Covers medical expenses for the rider and any passengers injured in an accident, 

Amazing! So what you have done so far is creating an agent, that consults the Azure AI Search Index that searches on your policies databases. Your query has made it such as it replies to your query with a text based solution. Let's print it out!